
# Querying astronomical databases with Astroquery

### ESOpy 3.0 @ Vitacura - 15-17 April 2019 - fvogt@eso.org



**Before we start:**
- Docs: https://astroquery.readthedocs.io/en/latest/

**Premise:** 
- lots of important online databases ... SIMBAD, NED, GAIA, NASA ADS, ESO archive ...

**Astroquery class:**
- a powerful way to access a wide range of astro databases using a consistent format
- **wrapper around existing webforms !!!**

**Cost:**
- 1 astropy sub-packages

**Benefits:**
- Get easy access to GAIA to anchor HST ACS images ...
- Find telescope guide stars ...
- Update your citation statistics (-> c.f. project)

**A note for the installation:**

- pip install --pre astroquery


## 1) Getting started

In [1]:
# Import the packages
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


## 2) Querying around a certain location

In [3]:
coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
radius = 30*u.arcsec
r = Gaia.query_object(coordinate=coord, radius=radius)

In [4]:
# Let's check the output
print(r.keys())
r.pprint()

['dist', 'solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'astrometric_weight_al', 'astrometric_pseudo_colour', 'astrometric_pseudo_colour_error', 'mean_varpi_factor_al', 'astrometric_matched_observations', 'visibility_periods_used', 'astrometric_sigma5d_max', 'frame_rotator_object_type', 'matched_observations', 'duplicated_source', 'phot_g_n_obs', 'phot_g_mean_flux', 'phot_g_mean_f

## 3) Practical break 1
Look at X1_Gaia-pm.py and try to fill the gaps!

## 4) Querying the ESO archive

In [9]:
import numpy as np
from astroquery.eso import Eso

In [11]:
# Connect to the archive
eso = Eso()
eso.ROW_LIMIT=-1 
eso.login("fvogt") 

fvogt, enter your password:
········
INFO: Authenticating fvogt on www.eso.org... [astroquery.eso.core]
INFO: Authentication successful! [astroquery.eso.core]


In [12]:
# What can be queried ?
eso.list_instruments()

['fors1',
 'fors2',
 'sphere',
 'vimos',
 'omegacam',
 'hawki',
 'isaac',
 'naco',
 'visir',
 'vircam',
 'apex',
 'giraffe',
 'uves',
 'xshooter',
 'espresso',
 'muse',
 'crires',
 'kmos',
 'sinfoni',
 'amber',
 'gravity',
 'midi',
 'pionier',
 'wlgsu']

In [17]:
# eso.query_instrument('muse', help=True)
table_muse = eso.query_instrument('muse', column_filters={'dp_cat':'SCIENCE',
                                                          'dp_type':'OBJECT',
                                                          #'dp_type':'SKY',
                                                          'stime':'2019-04-01', 
                                                          'etime':'2019-04-17',
                                                          #'prog_type':'1',
                                                          #'pi_coi':'VOGT',
                                                          },
                                                          #columns={},
                                                          )
table_muse.pprint()

Release Date      Object         RA     ... TPL EXPNO  INS MODE  DIMM Seeing-avg
------------ --------------- ---------- ... --------- ---------- ---------------
  2019-04-02    NGC5139_F136 201.712641 ...         1   NFM-AO-N     0.47 [0.01]
  2019-09-29           P1115 169.565833 ...         1   WFM-AO-N     0.57 [0.01]
  2019-09-29           P1115 169.575757 ...         2   WFM-AO-N     0.73 [0.01]
  2019-09-29           P1115 169.575755 ...         3   WFM-AO-N     0.74 [0.01]
  2020-04-02    SDSS J112023 170.099625 ...         1   WFM-AO-E     0.73 [0.01]
  2020-04-02    SDSS J112023 170.099769 ...         2   WFM-AO-E     0.70 [0.01]
  2020-04-02    SDSS J112023 170.099769 ...         3   WFM-AO-E     1.05 [0.30]
  2020-04-02        UGC 7342 184.580644 ...         2 WFM-NOAO-E     1.25 [0.57]
  2020-04-02        UGC 7342 184.580644 ...         3 WFM-NOAO-E     1.43 [0.79]
  2020-04-03       NGC  3201 154.519112 ...         1   WFM-AO-N     1.55 [0.94]
         ...             ...

In [18]:
print('Number of files: %i' % (len(table_muse)))
print(table_muse.keys())

Number of files: 330
['Release Date', 'Object', 'RA', 'DEC', 'Target Ra Dec', 'Target l b', 'ProgId', 'DP.ID', 'OB ID', 'OBS Target Name', 'EXPTIME [s]', 'DPR CATG', 'DPR TYPE', 'DPR TECH', 'TPL START', 'TPL NEXP', 'TPL EXPNO', 'INS MODE', 'DIMM Seeing-avg']


In [20]:
time_on_sky = np.sum(table_muse['EXPTIME [s]']) # Total time spent in that spot
print('Science time in P104: %.1f hours' % (time_on_sky/3600))

Science time in P104: 36.7 hours


## 5) Practical break 2
Look at X2_ESO-archive.py, and try to fill the gaps

## 6) Querying SIMBAD

In [48]:
from astroquery.simbad import Simbad

Simbad.add_votable_fields('flux(H)')

#Simbad.list_votable_fields()

result = Simbad.query_criteria('rah >15 & rah <16 & dec >-35 & dec < -29 & Hmag <15 & Hmag >10')

In [51]:
print(result.keys())
result['MAIN_ID', 'RA','DEC', 'FLUX_H'].pprint()

['MAIN_ID', 'RA', 'DEC', 'RA_PREC', 'DEC_PREC', 'COO_ERR_MAJA', 'COO_ERR_MINA', 'COO_ERR_ANGLE', 'COO_QUAL', 'COO_WAVELENGTH', 'COO_BIBCODE', 'FLUX_H']
         MAIN_ID                RA           DEC      FLUX_H
                             "h:m:s"       "d:m:s"     mag  
------------------------- ------------- ------------- ------
  2MASX J15025234-3425518 15 02 52.3397 -34 25 51.747 12.537
          TYC 7315-1707-1 15 05 38.0878 -33 38 16.126 10.411
           TYC 7319-178-1 15 01 50.0377 -34 57 00.986 10.587
           TYC 7319-187-1 15 07 33.3102 -34 18 00.728 10.089
           TYC 7319-267-1 15 07 23.8878 -33 50 07.801 10.778
           TYC 7319-346-1 15 07 57.0337 -34 59 26.341 10.289
           TYC 7319-357-1 15 04 51.4483 -34 13 54.980 11.046
           TYC 7319-771-1 15 02 41.5617 -34 16 28.389 10.242
           TYC 7319-836-1 15 07 39.6549 -34 42 37.044 10.124
               ESO 387-17  15 16 55.433  -34 28 21.77 11.473
                      ...           ...           ...  